In [15]:
import pandas as pd
import re

Задание №3

На основе уже знакомого тебе тестового набора данных из первого задания посчитай следующие метрики в тетрадке Jupyter 
Notebook:

1. Delivery rate

2. Open rate

3. Click to Open rate

4. Unsubscribe rate

5. Выяви лучшую тему

6. И лучший день недели (для планирования рассылок)


In [53]:
data = pd.read_csv('Data.csv')

In [54]:
df = data.drop(columns=['Название рассылки','Название кампании','Направление',
                        'Месяц', 'Год','Веб-версия','Номер недели',
                        'День недели', 'UTM Метка'])

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          218 non-null    datetime64[ns]
 1   day           218 non-null    object        
 2   time          218 non-null    object        
 3   Тема письма   218 non-null    object        
 4   Сегмент       218 non-null    object        
 5   sent          218 non-null    float64       
 6   done          218 non-null    float64       
 7   open          218 non-null    float64       
 8   click         218 non-null    float64       
 9   baunce        218 non-null    float64       
 10  retraction    218 non-null    float64       
 11  users         218 non-null    float64       
 12  step_1        218 non-null    int64         
 13  step_2        218 non-null    int64         
 14  step_3        218 non-null    int64         
dtypes: datetime64[ns](1), float64(7), int64(

In [ ]:
df = df.rename(columns={
                'Дата':'date',
                'День недели.1':'day',
                'Тема письма ':'theme',
                'Время':'time',
                'Сегмент ':'segment',
                'Отправлено':'sent',
                'Доставлено':'done',
                'Открытия':'open',
                'Клики':'click',
                'Баунсы (Все ошибки)':'baunce',
                'Отписки':'retraction',
                'Пользователей на сайте':'users',
                'Воронка продаж. Шаг 1':'step_1',
                'Воронка продаж. Шаг 2':'step_2',
                'Воронка продаж. Шаг 3':'step_3'
})

In [82]:
df.head(5)

,date,day,time,theme,segment,sent,done,open,click,baunce,retraction,users,step_1,step_2,step_3,dr,or,ctor,ur
0,2021-10-27,03-среда,19:24,Тема письма 1,Сегмент 1,780789.0,741750.0,148350.0,17802.0,39039.0,7417.0,16378.0,6337,6210,5154,95.0,20.0,12.0,1.0
1,2021-11-05,05-пятница,12:02,Тема письма 10,Сегмент 2,719370.0,683402.0,123012.0,11071.0,35969.0,6834.0,10296.0,3558,3096,2539,95.0,18.0,9.0,1.0
2,2022-04-11,01-понедельник,16:26,Тема письма 100,Сегмент 3,1201415.0,1141344.0,182615.0,15340.0,60071.0,11413.0,13959.0,3370,2864,2263,95.0,16.0,8.4,1.0
3,2022-04-12,02-вторник,16:26,Тема письма 101,Сегмент 1,1393827.0,1324136.0,264827.0,10328.0,69691.0,13241.0,8986.0,5116,4277,3208,95.0,20.0,3.9,1.0
4,2022-04-13,03-среда,16:26,Тема письма 102,Сегмент 2,1276821.0,1212980.0,218336.0,15720.0,63841.0,12130.0,10847.0,2817,2287,1670,95.0,18.0,7.2,1.0


In [ ]:
df.date = pd.to_datetime(df.date)

In [ ]:
# приведем колонки к числовым значениям и уберем лишние элементы 
for i in df.columns[5:]:    
   df[i] = df[i].apply(lambda x: (re.findall(r'\d+', x)))
   df[i] = df[i].apply(lambda x: (''.join(map(str, x))).replace(',',''))
   df[i] = df[i].astype(float)

In [76]:
# посчитаем метрики
df['dr'] = round((df.done / df.sent * 100), 2)       # Delivery rate
df['or'] = round((df.open / df.done * 100), 2)       # Open rate
df['ctor'] = round((df.click / df.open * 100), 2)    # Click to Open rate
df['ur'] = round((df.retraction / df.done * 100), 2) # Unsubscribe rate

In [77]:
df[['dr','or','ctor','ur']]

,dr,or,ctor,ur
0,95.0,20.00,12.0,1.0
1,95.0,18.00,9.0,1.0
2,95.0,16.00,8.4,1.0
3,95.0,20.00,3.9,1.0
4,95.0,18.00,7.2,1.0
...,...,...,...,...
213,98.5,17.43,3.9,0.9
214,98.0,11.05,7.2,0.9
215,98.0,9.20,12.0,0.9
216,98.5,11.05,9.0,0.9


#### Найдем лучшую тему. 

Считать будем по Click To Open Rate, т.к пользователь кликает, если тема письма интересная. В нашем случае у темы 1 и 101 одинаковый CTOR, поэтому примем за лучшую тему *тему 1*, потому что у нее выше OR. OR выбрал за вторую метрику, потому что пользователь на этапе получения письма читает заголовок, который имеет прямое отношение к теме. 

In [88]:
df[['theme', 'or', 'ctor','ur']].sort_values(['or','ctor'], ascending=False)

,theme,or,ctor,ur
0,Тема письма 1,20.0,12.0,1.00
3,Тема письма 101,20.0,3.9,1.00
55,Тема письма 149,18.0,12.0,0.59
65,Тема письма 158,18.0,12.0,0.59
135,Тема письма 24,18.0,12.0,13.00
...,...,...,...,...
68,Тема письма 160,9.2,3.9,0.50
133,Тема письма 22,9.2,3.9,0.15
143,Тема письма 31,9.2,3.9,13.00
148,Тема письма 36,9.2,3.9,13.00


#### Найдем самый лучшей день для отправки писем. 

Сгруппируем данные по дням недели. И отсортируем по OR и CTOR. В нашей табличке лучшим днем для рассылки является *среда*.

In [95]:
best_day = (df.groupby('day', as_index=False).agg({'dr':'mean', 'or':'mean', 'ctor':'mean', 'ur':'mean'})
   .sort_values(['or','ctor'], ascending=False ))

In [96]:
best_day

,day,dr,or,ctor,ur
2,03-среда,97.500000,14.326750,8.010000,1.419250
5,06-суббота,97.400000,13.796000,6.840000,0.554000
3,04-четверг,97.740741,13.701667,7.905556,1.819815
4,05-пятница,97.633333,13.656500,8.135000,3.151167
0,01-понедельник,97.911765,13.586765,8.144118,4.137647
1,02-вторник,97.400000,13.488800,8.976000,2.037200
